<a href="https://colab.research.google.com/github/wangqianyang/colab/blob/main/my_torrent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install dependencies

In [2]:
# 使用3.11的版本
!python --version

Python 3.11.13


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!python -m pip install --upgrade pip setuptools wheel && python -m pip install lbry-libtorrent wget torf && apt install python3-libtorrent
!python -m pip install --upgrade pip setuptools wheel && python -m pip install libtorrent wget torf && apt install python3-libtorrent

In [4]:
import libtorrent as lt
# 测试一下
print(f"成功导入 libtorrent 版本: {lt.version}")

成功导入 libtorrent 版本: 2.0.11.0


## 下载磁力链接

In [8]:
import libtorrent as lt
import time
import sys

# 创建一个 libtorrent 会话
ses = lt.session({'listen_interfaces': '0.0.0.0:6881'})

# 在这里输入您的磁力链接
#magnet_link = "magnet:?xt=urn:btih:..."
magnet_link = input("请输入您的磁力链接地址，并按回车键: ")

# 设置下载参数
params = lt.parse_magnet_uri(magnet_link)
params.save_path = '/content/drive/Shareddrives/shareDrives/download'  # 设置文件保存路径
h = ses.add_torrent(params)

print(f"开始下载: {h.name()}")

try:
    while not h.status().is_seeding:
        s = h.status()
        state_str = [
            '排队中', '检查文件中', '下载元数据中',
            '下载中', '完成', '做种中', '分配空间',
            '检查恢复数据'
        ]

        # 格式化进度输出
        progress_bar = '[' + '#' * int(s.progress * 20) + '-' * (20 - int(s.progress * 20)) + ']'

        print(
            f"\r{state_str[s.state]} {s.progress * 100:.2f}% "
            f"{progress_bar} "
            f"下载速度: {s.download_rate / 1000:.1f} kB/s | "
            f"上传速度: {s.upload_rate / 1000:.1f} kB/s | "
            f"连接节点: {s.num_peers} ",
            end=''
        )

        # 处理会话提醒
        alerts = ses.pop_alerts()
        for alert in alerts:
            if isinstance(alert, lt.metadata_received_alert):
                print("\n元数据接收完成，开始下载文件...")
            # 您可以在此处添加更多提醒处理逻辑
            # pass

        sys.stdout.flush()
        time.sleep(1)

    # 下载完成判断
    if h.status().is_seeding:
        print("\n下载完成！")


except KeyboardInterrupt:
    print("\n下载被中断。")

finally:
    # 暂停会话并保存恢复数据
    ses.pause()
    print("\n会话已暂停。")

请输入您的磁力链接地址，并按回车键: magnet:?xt=urn:btih:665B50D513A88B162129A208B57A1498215923A0&dn=Swiped+%282025%29+%5B720p%5D+%5BYTS.MX%5D&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce&tr=udp%3A%2F%2Fopen.tracker.cl%3A1337%2Fannounce&tr=udp%3A%2F%2Fp4p.arenabg.com%3A1337%2Fannounce&tr=udp%3A%2F%2Ftracker.torrent.eu.org%3A451%2Fannounce&tr=udp%3A%2F%2Ftracker.dler.org%3A6969%2Fannounce&tr=udp%3A%2F%2Fopen.stealth.si%3A80%2Fannounce&tr=udp%3A%2F%2Fipv4.tracker.harry.lu%3A80%2Fannounce&tr=https%3A%2F%2Fopentracker.i2p.rocks%3A443%2Fannounce
开始下载: Swiped (2025) [720p] [YTS.MX]
下载元数据中 0.00% [--------------------] 下载速度: 0.0 kB/s | 上传速度: 0.0 kB/s | 连接节点: 0 

/tmp/ipython-input-4193361127.py:17: DeprecationWarning: name() is deprecated
  print(f"开始下载: {h.name()}")


下载中 99.99% [###################-] 下载速度: 2478.5 kB/s | 上传速度: 127.6 kB/s | 连接节点: 119 
会话已暂停。


### Download torrent

In [ ]:
import zipfile
import os
import libtorrent as lt
import time
import datetime
import shutil

params = {
    'save_path': '/content/drive/Shareddrives/shareDrives/download',
    'storage_mode': lt.storage_mode_t(2),
}

ses = lt.session()
ses.listen_on(6881, 6891)
link = input("Input Torrent file link or Magnet link and Press Enter: ")

if link.endswith('.torrent'):
    import wget
    from torf import Torrent

    if os.path.exists('torrent.torrent'):
        os.remove('torrent.torrent')

    wget.download(link, 'torrent.torrent')
    t = Torrent.read('torrent.torrent')
    link = str(t.magnet(name=True, size=False, trackers=False, tracker=False))


print(link)
handle = lt.add_magnet_uri(ses, link, params)
# change the 0 to a 1 to download sequentially
handle.set_sequential_download(0)
ses.start_dht()
begin = time.time()

print(datetime.datetime.now())
print('Downloading Metadata...')

while not handle.has_metadata():
    time.sleep(1)

print('Got Metadata, Starting Torrent Download...')
print("Starting", handle.name())

while handle.status().state != lt.torrent_status.seeding:
    s = handle.status()
    state_str = ['queued', 'checking', 'downloading metadata',
                 'downloading', 'finished', 'seeding', 'allocating']
    print('%.2f%% complete (down: %.1f kb/s up: %.1f kB/s peers: %d) %s ' %
          (s.progress * 100, s.download_rate / 1000, s.upload_rate / 1000,
           s.num_peers, state_str[s.state]))
    time.sleep(5)

end = time.time()
print(handle.name(), "COMPLETE")
print("Elapsed Time: ", int((end - begin) // 60), "min :", int((end - begin) % 60), "sec")
print(datetime.datetime.now())

### Zip the files

In [ ]:
print('Zipping started...')

ziph = zipfile.ZipFile('Downloaded-Torrent.zip', 'w', zipfile.ZIP_DEFLATED)
path = '/content/drive/Shareddrives/shareDrives/download'
for root, dirs, files in os.walk(path):
    for file in files:
        ziph.write(os.path.join(root, file),
                   os.path.relpath(os.path.join(root, file),
                                   os.path.join(path, '..')))
ziph.close()

print('\nALL DONE!')